In [45]:
import pandas as pd

In [46]:
crimes = pd.read_csv('data/crime.csv')
education = pd.read_csv('data/education.csv')
population = pd.read_csv('data/population.csv')

### Cleaning Crimes Dataset and Joining to Education Dataset

In [47]:
# Removing the year 2021 as this is not included in the other datasets
crimes = crimes.loc[crimes['REF_DATE'] != 2021]

# Remove unwanted columns. I kept the 'GEO', 'REF_DATE', and 'VALUE' columns.
crimes = crimes[['GEO', 'REF_DATE', 'VALUE']]

# Group by 'GEO' and 'REF_DATE'. Here I summed together the values for each province for each year.
crimes_grouped = crimes.groupby(['GEO', 'REF_DATE'])['VALUE'].sum().reset_index()

# Clean 'GEO' names. Remove the number in brackets at the end of each province name.
crimes_grouped['GEO'] = crimes_grouped['GEO'].str.replace('\s*\[\d+\]', '', regex=True)

# Filter out unwanted rows. I removed the rows that contained the following value in the 'GEO' column.
crimes_grouped = crimes_grouped.loc[crimes_grouped['GEO'] != 'Canadian Forces Military Police']

# Rename columns. I renamed the columns to 'Province', 'Year', and 'Crimes'.
crimes_grouped = crimes_grouped.rename(columns={'VALUE': 'Crimes', 'GEO': 'Province', 'REF_DATE': 'Year'})

# Group Territories. I grouped the territories together and summed the values for each year.
territories = ['Northwest Territories', 'Nunavut', 'Yukon']
total_territories_crimes = crimes_grouped[crimes_grouped['Province'].isin(territories)].groupby('Year')['Crimes'].sum().reset_index()
total_territories_crimes['Province'] = 'Territories'

# Remove Territories from crimes_grouped and add total_territories_crimes to crimes_grouped.
crimes_grouped = pd.concat([crimes_grouped, total_territories_crimes], ignore_index=True)
crimes_grouped = crimes_grouped[~crimes_grouped['Province'].isin(territories)]

# Merging crimes_grouped with education based on Province and Year
merged_df = crimes_grouped.merge(education, on=['Province', 'Year'], how='outer')

# Merging the previous result with the population dataset based on Province and Year
final_df = merged_df.merge(population, on=['Province', 'Year'], how='outer')

In [48]:
final_df

,Province,Year,Crimes,Registrants,Population
0,Alberta,2017,1005325.43,194010.0,4258195.0
1,Alberta,2018,1043221.45,200391.0,4317665.0
2,Alberta,2019,1071487.39,203823.0,4384848.0
3,Alberta,2020,933926.17,203523.0,4421857.0
4,British Columbia,2017,838288.24,286914.0,4956550.0
5,British Columbia,2018,852692.63,295320.0,5040353.0
6,British Columbia,2019,949793.36,297432.0,5130780.0
7,British Columbia,2020,871957.23,287523.0,5151920.0
8,Manitoba,2017,361011.29,62802.0,1340565.0
9,Manitoba,2018,361049.00,63363.0,1357968.0
